In [ ]:
import sys
sys.path.insert(0, '..')

import matplotlib
import matplotlib.pyplot as plt
matplotlib.rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
matplotlib.rc('text', usetex=True)
matplotlib.rcParams.update({'font.size': 16})

import numpy as np
import pandas as pd
import pickle

from paus_utils import *
from jpasLAEs.utils import bin_centers

In [ ]:
### LOAD STUFF ###
# SDSS Xmatch
pathname = '/home/alberto/almacen/PAUS_data/catalogs/Xmatch_SDSS_W3.csv'
sdss_xm = pd.read_csv(pathname)
sdss_xm = sdss_xm.drop('Unnamed: 0', axis=1)
sdss_xm = sdss_xm.reset_index(drop=True)

# PAUS selection
region_list = ['W3']
sel_dfs = []
LFs_dir = '/home/alberto/almacen/PAUS_data/Lya_LFs'
for i in range(17):
    for region in region_list:
        with open(f'{LFs_dir}/Lya_LF_nb{i}-{i}_{region}/selection.pkl', 'rb') as file:
            this_dict = pickle.load(file)
            # Separate the L_lya err into two columns
            this_dict['L_lya_corr_err_up'] = this_dict['L_lya_corr_err'][1]
            this_dict['L_lya_corr_err_down'] = this_dict['L_lya_corr_err'][0]
            del this_dict['L_lya_corr_err']
        
        sel_dfs.append(pd.DataFrame(this_dict))
selection = pd.concat(sel_dfs)
# selection['ref_id'] = selection['ref_id'].astype(int)
selection = selection.drop_duplicates(subset='ref_id')
selection = selection.reset_index(drop=True)

# Join selection with sdss_xm
selection = pd.merge(selection, sdss_xm, on='ref_id', how='inner',
                     suffixes=['', '_SDSS'])

selection['z_NB'] = z_NB(selection['lya_NB'])

selection

In [ ]:
# Define nice_z
nice_z = np.array(np.abs(selection['z_CIV'] - z_NB(selection['lya_NB'])) < 0.2)

In [ ]:
# Plot EWs
fig, ax = plt.subplots(figsize=(6, 4))

log_EW_bins = np.linspace(1, 4, 20 + 1)
log_EW_bin_c = bin_centers(log_EW_bins)
ew_h, _ = np.histogram(np.log10(selection['EW0_lya']), log_EW_bins)
ew_h = ew_h / ew_h.sum()

ax.plot(log_EW_bin_c, ew_h, drawstyle='steps-mid',
        lw=2)

ax.set_ylabel('Fraction of sources', fontsize=16)
ax.set_xlabel(r'log$_{10}$ EW$_0^{\mathrm{Ly}\alpha}$ [\AA]', fontsize=16)

ax.tick_params(labelsize=14, direction='in', which='both')
ax.yaxis.set_ticks_position('both')
ax.xaxis.set_ticks_position('both')

ax.set_xlim(1, 4)

plt.show()

In [ ]:
# EW

fig, ax = plt.subplots(figsize=(6, 4))

mask = nice_z & (selection['EW0_lya_SDSS'] > 0)
ax.errorbar(selection['EW0_lya_SDSS'][mask], selection['EW0_lya'][mask],
           xerr=np.abs(selection['EW0_lya_err_SDSS'][mask]),
           yerr=selection['EW0_lya_err'][mask],
           fmt='o', ms=4, c='k', ecolor='k')

ax.plot([0.01, 1e6], [0.01, 1e6],
        ls='--', c='r', zorder=-99, label='1:1')

ax.set_yscale('log')
ax.set_xscale('log')

ax.set_xlim(3, 10000)
ax.set_ylim(10, 300)

ax.tick_params(labelsize=16, direction='in', which='both')
ax.yaxis.set_ticks_position('both')
ax.xaxis.set_ticks_position('both')

ax.set_ylabel('Narrow Band EW$_0$ [\AA]')
ax.set_xlabel('Spectroscopic EW$_0$ [\AA]')

plt.show()

In [ ]:
# L_Lya

fig, ax = plt.subplots(figsize=(6, 4))

mask = nice_z & (selection['L_lya_SDSS'] > 40)
ax.errorbar(selection['L_lya_SDSS'][mask], selection['L_lya'][mask],
            yerr=[selection['L_lya_corr_err_up'][mask],
                  selection['L_lya_corr_err_down'][mask]],
            xerr=[selection['L_lya_err_up'][mask],
                  selection['L_lya_err_down'][mask]],
            fmt='o', ms=3, c='k')

ax.plot([0.01, 1e6], [0.01, 1e6],
        ls='--', c='r', zorder=-99, label='1:1')

ax.set_xlim(42.9, 45.6)
ax.set_ylim(43.5, 45.5)

ax.tick_params(labelsize=16, direction='in', which='both')
ax.yaxis.set_ticks_position('both')
ax.xaxis.set_ticks_position('both')

ax.set_ylabel(r'Narrow Band $\log_{10}(L_{\mathrm{Ly}\alpha}/\mathrm{erg\,s}^{-1})$')
ax.set_xlabel(r'Spectroscopic $\log_{10}(L_{\mathrm{Ly}\alpha}/\mathrm{erg\,s}^{-1})$')

plt.show()


###############################################3

fig, ax = plt.subplots(figsize=(6, 4))

L_diff = 10**selection['L_lya'] - 10**selection['L_lya_SDSS']
L_diff_err_nb = 10**(selection['L_lya'] + selection['L_lya_corr_err_up']) - 10**selection['L_lya']
L_diff_err_sp = 10**(selection['L_lya_SDSS'] + selection['L_lya_err_up']) - 10**selection['L_lya_SDSS']
L_diff_err = (L_diff_err_nb ** 2 + L_diff_err_sp ** 2) ** 0.5

ax.hist((L_diff / L_diff_err)[mask], np.linspace(-5, 5, 20))
print(f'(mean, std) = {np.nanmean((L_diff / L_diff_err)[mask]):0.3f}, '
      f'{np.nanstd((L_diff / L_diff_err)[mask]):0.3f}')

plt.show()

In [ ]:
from jpasLAEs.utils import smooth_hist

sel_mask = (selection['z_CIV'] > 0) & (selection['L_lya'] > 44)
nice_mask = sel_mask & nice_z & (selection['L_lya'] > 44)

fig, ax = plt.subplots(figsize=(6, 4))

nice_h_smooth, to_plot_c = smooth_hist(selection['z_NB'][nice_mask],
                                       2.5, 4.5, 0.1, 0.4)
sel_h_smooth, to_plot_c = smooth_hist(selection['z_NB'][sel_mask],
                                      2.5, 4.5, 0.1, 0.4)
ax.plot(to_plot_c, nice_h_smooth / sel_h_smooth,
        lw=2)

ax.set_ylim(0, 1.01)

plt.show()